In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/small_cust.csv')

In [10]:
df.head()

,CustomerID,Recency,Frequency,Monetary
0,12821,215,6,92.72
1,12823,75,5,1759.50
2,12829,337,11,293.00
3,12831,263,9,215.05
4,12833,146,24,417.38


In [11]:
print("Shape", df.shape)
print("Frequency", sum(df.Frequency))
df.describe()

Shape (1330, 4)
Frequency 18968


,CustomerID,Recency,Frequency,Monetary
count,1330.000000,1330.000000,1330.000000,1330.000000
mean,15579.158647,162.975940,14.261654,284.403805
std,1609.583204,101.550624,10.738528,260.907356
min,12821.000000,2.000000,1.000000,3.750000
25%,14174.500000,68.000000,7.000000,142.400000
50%,15566.000000,160.000000,12.000000,223.865000
75%,16987.500000,250.000000,20.000000,344.710000
max,18282.000000,363.000000,77.000000,4055.720000


In [12]:
tr = pd.read_csv("data/retail.csv")

In [13]:
tf_f = tr[tr['CustomerID'].isin(df.CustomerID)]

In [15]:
print(tf_f.shape)
print("Frequency equals number of customers?", tf_f.shape[0] == sum(df.Frequency))

(18968, 8)
Frequency equals number of customers? True


In [16]:
tf_f['Description'] = tf_f['Description'].str.strip()
tf_f['InvoiceNo'] = tf_f['InvoiceNo'].astype(str)

/tmp/ipykernel_6976/2473024476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_f['Description'] = tf_f['Description'].str.strip()
/tmp/ipykernel_6976/2473024476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_f['InvoiceNo'] = tf_f['InvoiceNo'].astype(str)


In [17]:
basket = (tf_f.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

In [18]:
basket.drop('POSTAGE', inplace=True, axis=1)

In [20]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [21]:
def encode_units(x):
     if x <= 0:
         return 0
     if x >= 1:
         return 1

basket_sets = basket.applymap(encode_units)

/tmp/ipykernel_6976/1887240822.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.applymap(encode_units)


In [22]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

/home/codespace/.python/current/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [23]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [24]:
rules[ (rules['lift'] >= 6) &
        (rules['confidence'] >= 0.8) ].sort_values(['lift','confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
39,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.016356,0.031021,0.014664,0.896552,28.901567,0.014157,9.366798,0.981453
8,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.024253,0.031021,0.019741,0.813953,26.238901,0.018988,5.208263,0.985797
32,"(PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",(GREEN REGENCY TEACUP AND SAUCER),0.014664,0.031021,0.011844,0.807692,26.037063,0.011389,5.038691,0.975904
2,(BAKING SET SPACEBOY DESIGN),(BAKING SET 9 PIECE RETROSPOT),0.019741,0.042865,0.015792,0.800000,18.663158,0.014946,4.785674,0.965478
0,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023689,0.050197,0.019177,0.809524,16.126806,0.017987,4.986464,0.960750
